In [ ]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread("img/paper.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
win_name = "result"
print(img.shape)
emptyBack = np.full(img.shape, 255, np.uint8)

dst = cv2.GaussianBlur(img_gray, (5,5),0) 
edges = cv2.Canny(dst,100,200)

# 컨투어들 중에 영역 크기 순으로 정렬

ret, contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(hierarchy)

for cnt in contours:

    epsilon = 0.005 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    print(approx[:][:][0])
    
    vertices = len(approx)
    
    print("vertices:", vertices)
    points = np.array(approx)
    poly = cv2.polylines(emptyBack, [points], True, (0, 0, 255), 4)
    
    # 중심점 찾기
    mmt = cv2.moments(cnt)
    cx,cy = int(mmt['m10']/mmt['m00']), int(mmt['m01']/mmt['m00'])


# 좌표 4개 중 상하좌우 찾기 ---② 

j = 0
k = 0

for i in approx:
    if (cx > i[j][k]) and (cy > i[j][k+1]):
        ulpt = [i[j][k], i[j][k+1]]
    
    elif (cx > i[j][k]) and (cy < i[j][k+1]):
        dlpt = [i[j][k], i[j][k+1]]
        
    elif (cx < i[j][k]) and (cy > i[j][k+1]):
        urpt = [i[j][k], i[j][k+1]]

    elif (cx < i[j][k]) and (cy < i[j][k+1]):
        drpt = [i[j][k], i[j][k+1]]
        
        
print(ulpt,dlpt,urpt,drpt)




# 변환 전 4개 좌표 

orig_pt = np.float32([ulpt,dlpt,urpt,drpt])



# 변환 후 영상에 사용할 서류의 폭과 높이 계산 ---③ 

rows = img.shape[0]
cols = img.shape[1]

after_pt = np.float32([[0,0], [0,rows], [cols, 0], [cols,rows]])
mtrx = cv2.getPerspectiveTransform(orig_pt, after_pt)




# 변환 후 4개 좌표

result = cv2.warpPerspective(img, mtrx, (cols, rows))

result = cv2.rectangle(result, (0, 0), (cols, rows), (255, 0, 0), 5)

# cv2.imshow("dst", dst)
cv2.imshow("edges", edges)
cv2.imshow("contour", poly)
cv2.imshow("result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

(782, 586, 3)
[[[-1 -1 -1 -1]]]
[[101  55]]
vertices: 4
[101, 55] [23, 706] [497, 103] [568, 695]
